In [10]:
import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
import dash
import dash
from dash import dcc
from dash import html
import plotly.express as px
import csv
import os
import pathlib as path
import plotly #https://plotly.com/python/getting-started/ /////// pip install plotly

In [7]:

toplaner_agg = path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\TopLaneStats_agg_50games_majorLeague.csv')
toplaner_agg_winloss =  path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\TopLaneStats_agg_Win-Loss_majorLeague.csv')
df_ta = pd.read_csv(toplaner_agg)
df_wl = pd.read_csv(toplaner_agg_winloss)

In [13]:
app = dash.Dash(__name__)
fig = px.bar(df_ta, x='player',y='avg(earnedgoldshare)', color='wins', barmode ='group')

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),
    html.Div(children='''Hello Dash')''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\Python310\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

